<a href="https://colab.research.google.com/github/cmari038/Language-Translator/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install spacy
#!pip install collections

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

In [ ]:
import torch
import torchtext; torchtext.disable_torchtext_deprecation_warning()
import pandas as pd
import numpy as np
import spacy
import torch.nn as nn
import torch.nn.functional as functional
from torchtext.data.utils import get_tokenizer
from torch.utils.data import random_split
from torchtext.vocab import vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter, OrderedDict

In [ ]:
# connect pytorch to cuda
device = torch.device("cuda")

dataset = 'https://raw.githubusercontent.com/cmari038/Language-Translator/main/data.csv'
data = pd.read_csv(dataset)

#spacy.load('en_core_web_sm')
#spacy.load('es_core_news_sm')

# processing data

english_tokenizer = get_tokenizer('spacy', language = 'en_core_web_sm')
spanish_tokenizer = get_tokenizer('spacy', language= 'es_core_news_sm')

#print(data)

#train = data.sample(frac=0.7)
#validate = data.drop(train.index).sample(frac=0.1)
#test = data.drop(validate.index)

train = data.iloc[:int(len(data) * 0.80)]

en_lowercase = []
for element in train['english']:
  en_lowercase.append(element.lower())

train.loc[:, 'english'] = en_lowercase

es_lowercase = []
for element in train['spanish']:
  es_lowercase.append(element.lower())

train.loc[:,'spanish'] = es_lowercase

#train.reset_index(drop=True, inplace=True)
#validate.reset_index(drop=True, inplace=True)

# Build vocab structure to store tokens and their corresponding index
counter1 = Counter()
counter2 = Counter()

for sentence in train['english']:
  counter1.update(english_tokenizer(sentence))

for sentence in train['spanish']:
  counter2.update(spanish_tokenizer(sentence))

en_dict = OrderedDict(counter1.most_common())
es_dict = OrderedDict(counter2.most_common())

vocab1 = vocab(en_dict, specials = ['<unk>', '<pad>', '<sos>', '<eos>'])
vocab2 = vocab(es_dict, specials = ['<unk>', '<pad>', '<sos>', '<eos>'])

vocab1.set_default_index(vocab1['<unk>'])
vocab2.set_default_index(vocab2['<unk>'])

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [155]:
print(train)

118964


In [ ]:
# finding set of tokens for each phrase
def getTokens(df, en_tokenizer, es_tokenizer, lang):
  if lang == "english":
    en_tokens = []

    for token in en_tokenizer(df):
      en_tokens.append(token)

    en_tokens = ['<sos>'] + en_tokens + ['<eos>']

    return en_tokens

  else:
    es_tokens = []

    for token in es_tokenizer(df):
        es_tokens.append(token)

    es_tokens = ['sos'] + es_tokens + ['<eos>']

    return es_tokens


#token_dict = {"en_tokenizer": english_tokenizer, "es_tokenizer": spanish_tokenizer}
#train = train.apply(map(lambda col: col.map(getTokens)))
en_tokens = []
es_tokens = []

for element in train.loc[:,'english']:
  tmp1 = getTokens(element, english_tokenizer, spanish_tokenizer, "english")
  en_tokens.append(tmp1)

for element in train.loc[:,"spanish"]:
  tmp2 = getTokens(element, english_tokenizer, spanish_tokenizer, "spanish")
  es_tokens.append(tmp2)

train['en_tokens'] = en_tokens
train['es_tokens'] = es_tokens

#print(train['en_tokens'])

<ipython-input-142-4fdc740c78c6>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['en_tokens'] = en_tokens
<ipython-input-142-4fdc740c78c6>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['es_tokens'] = es_tokens


In [ ]:
print(train['en_tokens'])

0                                    [<sos>, go, ., <eos>]
1                                    [<sos>, go, ., <eos>]
2                                    [<sos>, go, ., <eos>]
3                                    [<sos>, go, ., <eos>]
4                                    [<sos>, hi, ., <eos>]
                               ...                        
95166    [<sos>, i, thought, you, 'd, wear, something, ...
95167    [<sos>, i, threw, the, strange, package, on, t...
95168    [<sos>, i, told, tom, mary, would, n't, accept...
95169    [<sos>, i, told, tom, that, he, could, n't, sp...
95170    [<sos>, i, told, her, what, he, was, doing, in...
Name: en_tokens, Length: 95171, dtype: object


In [ ]:
def getIndices(df, en_vocab, es_vocab, lang):
  if lang == "english":
    en_indices = []

    for word in df:
      en_indices.append(en_vocab[word])

    return en_indices

  else:
    es_indices = []

    for word in df:
        es_indices.append(es_vocab[word])

    return es_indices

en_indices = []
es_indices = []

for element in train.loc[:,"en_tokens"]:
  tmp1 = getIndices(element, vocab1, vocab2, "english")
  en_indices.append(tmp1)

for element in train.loc[:, "es_tokens"]:
  tmp1 = getIndices(element, vocab1, vocab2, "spanish")
  es_indices.append(tmp1)

train['es_indices'] = es_indices
train['en_indices']= en_indices

<ipython-input-144-49ececf8f000>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['es_indices'] = es_indices
<ipython-input-144-49ececf8f000>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['en_indices']= en_indices


"\nprint(train['en_indices'])\n\nfor series in train['en_indices']:\n  en_tensor.append(torch.tensor(series, dtype=torch.long))\n\nfor series in train['es_indices']:\n  es_tensor.append(torch.tensor(series, dtype=torch.long))\n\ntrain['en_indices'] = en_tensor\ntrain['es_indices'] = es_tensor\n"

In [ ]:
print(train['en_indices'])

0                    [2, 15, 53, 37, 4434, 7, 85, 8, 4, 3]
1                       [2, 27, 21, 1524, 7, 6, 994, 4, 3]
2           [2, 5, 146, 8, 121, 9, 71, 101, 398, 32, 4, 3]
3        [2, 5, 286, 39, 7, 150, 102, 108, 2890, 263, 4...
4          [2, 255, 240, 6010, 22, 27, 17, 60, 1525, 4, 3]
                               ...                        
83270                    [2, 8, 23, 7, 220, 18, 375, 4, 3]
83271           [2, 12, 25, 6, 311, 28, 442, 11741, 11, 3]
83272    [2, 75, 10, 2588, 47, 14, 16, 22, 10, 578, 47,...
83273    [2, 130, 2484, 185, 4930, 3231, 440, 22, 102, ...
83274                               [2, 47, 29, 50, 11, 3]
Name: en_indices, Length: 83275, dtype: object


In [149]:
# convets dataframe to dataset for tensor acess
class TensorSet(Dataset):
  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    return torch.tensor(self.df['en_indices'].iloc[index], dtype=torch.long), torch.tensor(self.df['es_indices'].iloc[index], dtype=torch.long)

def collate_fn(batch):
  # used for making sure sequences are similar lengths by adding tokens to pad out the length
  en_batch = []
  es_batch = []
  for en_sample, es_sample in batch:
    en_batch.append(en_sample)
    es_batch.append(es_sample)

  en_batch = pad_sequence(en_batch, padding_value=vocab1['<pad>'])
  es_batch = pad_sequence(es_batch, padding_value=vocab2['<pad>'])

  return en_batch, es_batch

#tensorSet = TensorSet(train, english_tokenizer, spanish_tokenizer, vocab1, vocab2)
tensorSet = TensorSet(train)
batch_size = 128
dataLoad = DataLoader(tensorSet, shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
#evalDataLoad = DataLoader(TensorSet(validate), shuffle=True, collate_fn=collate_fn, batch_size=batch_size)

In [152]:
# RNN model
# made up of encoder, deocder, and seq2seq
# encoder converts tensors into context vectors
class RNN_Encoder(nn.Module):
    def __init__(self, input, embedding_dimension, hidden_dimension, layers, dropout_p=0.5):
        super(RNN_Encoder, self).__init__()
        self.hidden_dimension = hidden_dimension
        self.layers = layers
        self.embed = nn.Embedding(input, embedding_dimension)
        self.lstm = nn.LSTM(embedding_dimension, hidden_dimension, layers, dropout=dropout_p)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, english):
        embedded = self.dropout(self.embed(english))
        output, (hidden, cell) = self.lstm(embedded)
        return hidden, cell

# decoder uses context vectors to output spanish translation
class RNN_Decoder(nn.Module):
    def __init__(self, output, embedding_dimension, hidden_dimension, layers, dropout_p=0.5):
      super(RNN_Decoder, self).__init__()
      self.output = output
      self.hidden_dimension = hidden_dimension
      self.layers = layers
      self.embed = nn.Embedding(output, hidden_dimension)
      self.lstm = nn.LSTM(embedding_dimension, hidden_dimension, layers, dropout=dropout_p)
      self.fc_out = nn.Linear(hidden_dimension, output)
      self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden, cell):
      input = input.unsqueeze(0)
      embedded = self.dropout(self.embed(input))
      out, (hidden, cell) = self.lstm(embedded, (hidden, cell))
      spanish_prediction = self.fc_out(out.squeeze(0))
      #output = self.embed(input)
      #output = functional.relu(output)
      #output, hidden = self.gru(output, hidden)
      return spanish_prediction, hidden, cell

# sequence to sequence that ties encoder and decoder together
class Sequence(nn.Module):
    def __init__(self, encoder, decoder, device):
      super(Sequence, self).__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.device = device

    def forward(self, english, spanish, teacher_forcing_ratio):
      batch_size = spanish.shape[1]
      es_length = spanish.shape[0]
      es_vocab_size = self.decoder.output
      outputs = torch.zeros(es_length, batch_size,es_vocab_size).to(self.device)
      hidden, cell = self.encoder(english)
      input = spanish[0,:]

      for i in range(1, es_length):
        output, hidden, cell = self.decoder(input, hidden, cell)
        outputs[i] = output
        teacher_force = np.random.random() < teacher_forcing_ratio
        top1 = output.argmax(1)
        decoder_input = spanish[i] if teacher_force else top1

      return outputs

encoder = RNN_Encoder(input=len(vocab1), embedding_dimension=256, hidden_dimension=256, layers=2)
decoder = RNN_Decoder(output=len(vocab2), embedding_dimension=256, hidden_dimension=256, layers=2)
RNN_model = Sequence(encoder, decoder, device).to(device)

In [158]:
criterion = nn.CrossEntropyLoss(ignore_index=vocab2['<pad>'])
optimizer = torch.optim.Adam(RNN_model.parameters())

# defines weights
def weights(model):
  for name, parameter in model.named_parameters():
    nn.init.uniform_(parameter.data, -0.5, 0.5)

def training(RNN_model, dataLoad, criterion, optimizer, device, epochs, teacher_forcing_ratio, clip):
  #RNN_model.train()
  for epoch in range(epochs):
    RNN_model.train()
    epoch_loss = 0
    for (english, spanish) in dataLoad: # iterate in batches of size 128 through dataloader
      english = english.to(device)
      spanish = spanish.to(device)
      optimizer.zero_grad()
      # passing tensors into neural network
      output = RNN_model(english, spanish, teacher_forcing_ratio)
      output = output[1:].view(-1, output.shape[-1])
      spanish = spanish[1:].view(-1)
      # compares neural network's spanish translation to translation from dataset
      loss = criterion(output, spanish)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(RNN_model.parameters(),clip)
      optimizer.step()
      epoch_loss += loss.item()

RNN_model.apply(weights)
training(RNN_model, dataLoad, criterion, optimizer, device, 5, 0.5, 1.0)

In [171]:
# translates an input sentece into spanish using trained model
def translation(model, input, vocab1, vocab2, device):
  input = input.lower()
  #print(input)
  model.eval()
  #tokens = english_tokenizer(input)
  #indices = []
  """for token in tokens:
      indices.append(vocab1[token])
  input_tensor = torch.tensor([vocab1['<sos>']] + indices + [vocab1['<eos>']], dtype=torch.long) """

  with torch.no_grad():
    tokens = getTokens(input, english_tokenizer, spanish_tokenizer, "english")
    indices = getIndices(tokens, vocab1, vocab2, "english")
    #print(indices)
    tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(-1).to(device)
    #print(tensor)
    hidden, cell = model.encoder(tensor)
    spanish = [vocab2['<sos>']]

    """encoder_output, hidden = model.encoder(tensor, len(tokens))
    spanish_vocab = {i: word for word, i in spanish_vocab.items()}
    decoder_input = torch.tensor([vocab2['<sos>']]).to(device)
    spanish = [] """

    for i in range(50):
        tensorInput = torch.tensor([spanish[-1]], dtype=torch.long).to(device)
        decoder_output, hidden, cell = model.decoder(tensorInput, hidden, cell)
        top1 = decoder_output.argmax(-1).item()
        if top1 == vocab2['<eos>']:
          spanish.append(top1)
          break
        spanish.append(top1)
        decoder_input = torch.tensor([top1]).to(device)
    tokens = vocab2.lookup_tokens(spanish)
  return tokens


#train(RNN_model, dataLoad, criterion, optimizer, device, 10, 0.5)
# I am taking my car to the library.
# My sister is my best friend and we enjoy playing games together.
# Hi there! How was your day?
# My dog is running towards me.
# I want to go to the beach.
#sentence = " I want to go to the beach."
#translated_sentence = translation(RNN_model, sentence, vocab1, vocab2, device)
#print(translated_sentence)

In [174]:
sentence = "I want to go to the beach."
translated_sentence = translation(RNN_model, sentence, vocab1, vocab2, device)
print(translated_sentence)

['<sos>', 'quiero', 'quiero', 'en', 'la', '.', '.', '.', '<eos>']


In [175]:
sentence = "I am taking my car to the library"
translated_sentence = translation(RNN_model, sentence, vocab1, vocab2, device)
print(translated_sentence)

['<sos>', 'estoy', 'un', 'mi', 'de', 'en', 'mi', '.', '.', '.', '.', '.', '<eos>']


In [176]:
sentence = "My sister is my best friend and we enjoy playing games together."
translated_sentence = translation(RNN_model, sentence, vocab1, vocab2, device)
print(translated_sentence)

['<sos>', 'mi', 'madre', 'que', 'no', ',', 'que', 'de', 'un', 'un', 'un', '.', '.', '.', '.', '.', '.', '.', '.', '.', '<eos>']


In [177]:
sentence = "Hi there! How was your day?"
translated_sentence = translation(RNN_model, sentence, vocab1, vocab2, device)
print(translated_sentence)

['<sos>', '¿', 'alguna', '¿', 'alguna', 'es', 'alguna', 'tuyos', 'este', 'su', 'ella', 'a', '<eos>']


In [178]:
sentence = "My dog is running towards me."
translated_sentence = translation(RNN_model, sentence, vocab1, vocab2, device)
print(translated_sentence)

['<sos>', 'mi', 'mi', 'es', 'para', 'se', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '<eos>']
